In [60]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [62]:
cie_10 = pd.read_excel("../data/external/CIE-10 - sin_puntos_y_X.xlsx")
regiones = (
    pd.read_excel("../data/external/Esquema_Registro-2023.xlsx", sheet_name=2, header=6)
    .dropna(how="all", axis=1)
    .iloc[:-1, :-1]
)
regiones["Código Comuna"] = regiones["Código Comuna"].astype(int)
regiones["Código Región"] = regiones["Código Región"].astype(int)

## 1. Análisis Sociodemografico a lo largo del tiempo

En este analisis se quieren responder las siguientes preguntas:

1. ¿Cuántas consultas hay por año?
2. ¿Cuáles son los diagnósticos más frecuentes por año?
3. ¿Cuál es el **rango etario** de las personas que se atienden ambulatoriamente en el Hospital del
Tórax? ¿Cómo cambia este perfil por cada uno de los diagnóstico?
4. ¿Cuál es el **género/sexo** de las personas que se atienden ambulatoriamente en el Hospital del
Tórax? ¿Cómo cambia este perfil por cada uno de los diagnósticos?
5. ¿Cuál es la **previsión** de las personas que se atienden ambulatoriamente en el Hospital del
Tórax? ¿Cómo cambia este perfil por cada uno de los diagnósticos?
6. ¿Cuál es el **servicio de salud** de las personas que se atienden ambulatoriamente en el Hospital del
Tórax? ¿Cómo cambia este perfil por cada uno de los diagnósticos?
7. ¿Cuál es la **comuna** de las personas que se atienden ambulatoriamente en el Hospital del
Tórax? ¿Cómo cambia este perfil por cada uno de los diagnósticos?

Para responder estas preguntas se utilizará la base de datos de consultas ambulatorias
del sistema HIS del Instituto Nacional del Tórax. La base de datos contiene consultas desde
el año 2016 al 2022. Esta base contiene los datos para saber la frecuencia por diagnóstico,
rango etario y género.

A pesar de lo anterior, la base de datos de consultas carece de los últimos 3 datos a analizar
(previsón, servicio de salud y comuna de residencia). Debido a esta razón, se utilizará la base
de TrackCare para obtener tales datos.

Además, se quiere responder cada una de estas preguntas para cada una de las especialidades de
las consultas, diagnósticos y por tipo de consulta.


In [63]:
df_track = pd.read_csv("../data/processed/datos_limpios_track.csv", sep=";", encoding="latin-1")
df_his = pd.read_csv(
    "../data/processed/datos_limpios_diagnosticos.csv", sep=";", encoding="latin-1"
)

In [64]:
df_track["hora_completa_cita"] = pd.to_datetime(df_track["hora_completa_cita"])
df_his["fecha_reserva"] = pd.to_datetime(df_his["fecha_reserva"])

In [65]:
print(
    f"La base de Track tiene citas agendadas desde {df_track['hora_completa_cita'].min()} "
    f"a {df_track['hora_completa_cita'].max()}"
)

print(
    f"La base de HIS tiene citas agendadas desde {df_his['fecha_reserva'].min()} a "
    f"{df_his['fecha_reserva'].max()}"
)
print()
print(f"La base de Track tiene {df_track.shape[0]} citas")
print(f"La base de HIS tiene {df_his.shape[0]} citas")

La base de Track tiene citas agendadas desde 2016-01-04 08:00:00 a 2022-12-30 20:30:00
La base de HIS tiene citas agendadas desde 2016-01-04 08:00:00 a 2022-12-30 14:50:00

La base de Track tiene 758350 citas
La base de HIS tiene 265441 citas


In [66]:
datos_pacientes_unicos_por_sesion = df_track.groupby(["id_paciente", "hora_completa_cita"]).head(1)

In [67]:
df_his_y_track = pd.merge(
    df_his,
    datos_pacientes_unicos_por_sesion,
    how="left",
    left_on=["id_paciente", "fecha_reserva"],
    right_on=["id_paciente", "hora_completa_cita"],
)

sin_datos_en_track = df_his_y_track[df_his_y_track.estadocita.isna()]

In [68]:
print(
    f"Luego de unir la base de HIS con Track, la primera ha quedado con "
    f"{df_his_y_track.shape[0]} consultas (de {df_his.shape[0]}). "
    f"Hay {sin_datos_en_track.shape[0]} consultas de HIS sin información en Track"
)

Luego de unir la base de HIS con Track, la primera ha quedado con 265441 consultas (de 265441). Hay 189 consultas de HIS sin información en Track


In [69]:
for paciente in sin_datos_en_track.id_paciente.unique():
    if paciente in df_track["id_paciente"]:
        print(f"{paciente} si esta")

Los resultados anteriores indican que ninguno de los pacientes faltantes se encuentra en la base
de Track (Y no ocurre debido a que los pacientes existen en la base, pero tienen una fecha de atencion distinta a la de HIS). 
Debido a lo anterior, estos pacientes carecen de datos sociodemográficos.

Una vez realizado la unión de las bases de datos es posible realizar el conteo anidado por
las variables sociodemográficas.

In [79]:
VARIABLES_SOCIODEMOGRAFICAS_INTERES = [
    "ano",
    "sexo_x",
    "comuna",
    "provincia",
    "region",
    "plan",
    "prevision",
    "tipoatencion",
    "rango_etario",
]

VARIABLE_A_CONTAR = ["nombre_diagnostico", "codigo_diagnostico"]

In [87]:
resultado_sociodemografico = (
    df_his_y_track.groupby(VARIABLES_SOCIODEMOGRAFICAS_INTERES)[VARIABLE_A_CONTAR]
    .value_counts()
    .reset_index(name="cantidad_de_consultas")
)

resultado_sociodemografico["comuna"] = resultado_sociodemografico["comuna"].str.upper()
resultado_sociodemografico = pd.merge(
    resultado_sociodemografico, regiones, how="left", left_on="comuna", right_on="Nombre Comuna"
).merge(cie_10, how="left", left_on="codigo_diagnostico", right_on="Código")

resultado_sociodemografico = resultado_sociodemografico.rename(
    columns={"Código Región": "codregion", "Código Comuna": "cod_comuna"}
)

resultado_sociodemografico["codregion"] = resultado_sociodemografico["codregion"].astype("Int32")
resultado_sociodemografico["cod_comuna"] = resultado_sociodemografico["cod_comuna"].astype("Int32")

In [88]:
resultado_sociodemografico.to_csv(
    "../data/interim/sociodemografico/desglose_sociodemografico.csv",
    index=False,
    encoding="latin-1",
    sep=";",
)